In [45]:
from modules.imports import *
from modules.metrics_here import *
from modules.api import *

In [102]:
# Importing dataset
stockcode = 'TCS'
data = yf.download(tickers=stockcode+'.NS', period='max', interval='1d').reset_index()

[*********************100%***********************]  1 of 1 completed


In [103]:
## Marking potential high points

# Checking if the high was the hjighest in the past 30 days
data['r_max'] = data['High'].rolling(30).max()
data['diff'] = data['High'] - data['r_max']

data['reached_high'] = 0
data['reached_high'].loc[data['diff'] == 0] = 1

# Should not be the high in the next 5 days
data['max_yesterday'] = data['reached_high']
for i in range(1,6):
    data['max_yesterday'] = data['reached_high'].shift(-1*i).fillna(0) + data['max_yesterday']

data['reached_high_cont5'] = 0
data['reached_high_cont5'].loc[(data['max_yesterday'] == 1) & (data['reached_high'] == 1)] = 1


# Dropping unwanted columns
data = data.drop(['max_yesterday', 'diff', 'r_max', 'reached_high'], axis = 1)

In [104]:
# Getting out 2 consecutive red candles
data['temp'] = data['Open'] - data['Close']
data['red_candle'] = 0
data['red_candle'].loc[data['temp'] > 0] = 1
del data['temp']

data['red_candle_lag1'] = data['red_candle'].shift(-1)
data['red_candle_lag2'] = data['red_candle'].shift(-2)
data['temp'] = data['reached_high_cont5'] + data['red_candle_lag1'] + data['red_candle_lag2']

# Generating event dates
data['event'] = 0
data['event'].loc[data['temp'] == 3] = 1

# Dropping unwanted columns
data = data.drop(['reached_high_cont5', 'red_candle', 'red_candle_lag1', 'red_candle_lag2', 'temp'], axis = 1)

In [108]:
# Point of inflection

# Finding out points whose lowest are from the past and next 2 days

data['future_low1'] = data['Low'].shift(-1)
data['future_low2'] = data['Low'].shift(-2)
data['past_low1'] = data['Low'].shift(1)
data['past_low2'] = data['Low'].shift(2)
data['local_min_low'] = data[['past_low1','past_low2', 'future_low1', 'future_low2', 'Low']].min(axis=1)
# Generating low inflection points
data['low_inflection'] = 0
data['low_inflection'].loc[data['Low'] == data['local_min_low']] = 1

# # Closing inflection
# data['future_close1'] = data['Close'].shift(-1)
# data['future_close2'] = data['Close'].shift(-2)
# data['past_close1'] = data['Close'].shift(1)
# data['past_close2'] = data['Close'].shift(2)
# data['local_min_close'] = data[['past_close1','past_close2', 'future_close1', 'future_close2', 'Close']].min(axis=1)
# # Generating close inflection points
# data['close_inflection'] = 0
# data['close_inflection'].loc[data['Close'] == data['local_min_close']] = 1

# # Selecting only those points with both low and close inflection
# data['inflection'] = 0
# data['inflection'].loc[(data['close_inflection'] == 1) & (data['low_inflection'] == 1)] = 1

data = data.drop(['future_low1', 'future_low2', 'past_low1', 'past_low2', 'local_min_low'], axis = 1)

In [109]:
data[data['event'] == 1]

,Date,Open,High,Low,Close,Adj Close,Volume,event,low_inflection
74,2002-11-22,41.387501,45.174999,41.387501,42.337502,31.419977,1184016,1,0
103,2003-01-02,61.525002,62.924999,57.912498,58.299999,43.266243,5325328,1,0
511,2004-07-28,100.500000,103.000000,99.000000,101.062500,75.001640,182768,1,0
523,2004-08-13,110.000000,111.750000,105.000000,106.875000,79.315247,119976,1,0
531,2004-08-25,149.837494,149.837494,122.375000,123.493752,91.648537,136928,1,0
...,...,...,...,...,...,...,...,...,...
4554,2020-12-09,2815.000000,2840.000000,2793.149902,2810.800049,2780.390869,3659996,1,0
4586,2021-01-25,3308.949951,3339.800049,3278.649902,3291.300049,3261.887695,2272887,1,0
4636,2021-04-09,3354.000000,3354.350098,3308.000000,3322.250000,3292.561279,2911129,1,0
4688,2021-06-25,3375.250000,3399.649902,3350.000000,3380.800049,3366.977539,2722861,1,0


In [113]:
data[data['Date'] >= '2021-06-25'].head(30)

,Date,Open,High,Low,Close,Adj Close,Volume,event,low_inflection
4688,2021-06-25,3375.250000,3399.649902,3350.000000,3380.800049,3366.977539,2722861,1,0
4689,2021-06-28,3391.500000,3391.500000,3333.000000,3336.750000,3323.107666,1355238,0,0
4690,2021-06-29,3342.000000,3366.300049,3326.500000,3340.850098,3327.191162,1497322,0,1
4691,2021-06-30,3346.000000,3382.000000,3338.500000,3345.750000,3332.070801,1693146,0,0
4692,2021-07-01,3358.000000,3362.000000,3332.350098,3341.500000,3327.838379,990310,0,0
4693,2021-07-02,3331.000000,3339.699951,3295.199951,3325.449951,3311.853760,1682274,0,0
4694,2021-07-05,3342.000000,3374.000000,3302.000000,3321.000000,3307.422119,1972858,0,0
4695,2021-07-06,3313.000000,3328.000000,3256.100098,3262.300049,3248.961914,2159420,0,0
4696,2021-07-07,3275.000000,3282.699951,3241.600098,3274.949951,3261.560059,2640710,0,0
4697,2021-07-08,3298.000000,3298.000000,3231.899902,3258.500000,3245.177490,2485811,0,0


In [92]:
data.tail(30)

,Date,Open,High,Low,Close,Adj Close,Volume,event,inflection
4788,2021-11-22,3478.399902,3515.949951,3440.300049,3458.399902,3458.399902,3167936,0,0
4789,2021-11-23,3445.000000,3480.949951,3407.800049,3464.250000,3464.250000,2153461,0,0
4790,2021-11-24,3472.199951,3493.050049,3424.000000,3443.300049,3443.300049,2161746,0,0
4791,2021-11-25,3443.899902,3463.000000,3434.050049,3445.899902,3445.899902,1849619,0,0
4792,2021-11-26,3425.000000,3490.000000,3411.899902,3446.850098,3446.850098,1941251,0,0
4793,2021-11-29,3447.149902,3529.350098,3406.449951,3502.000000,3502.000000,2981745,0,0
4794,2021-11-30,3491.000000,3563.750000,3491.000000,3529.149902,3529.149902,5588446,0,0
4795,2021-12-01,3535.000000,3590.000000,3535.000000,3577.800049,3577.800049,2102118,0,0
4796,2021-12-02,3589.199951,3648.000000,3574.100098,3642.899902,3642.899902,3152938,0,0
4797,2021-12-03,3646.449951,3665.949951,3630.850098,3640.449951,3640.449951,2486263,0,0


In [67]:
fig = go.Figure()

#Candlestick
fig.add_trace(go.Candlestick(x=data['Date'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'], name = 'market data'))

temp = data[data['event'] == 1]

fig.add_trace(go.Scatter(x=temp['Date'],y=temp['High']))

fig.update_xaxes(
    rangeslider_visible=True,
    rangebreaks=[{ 'pattern': 'day of week', 'bounds': [6, 1]}
    ]
)

# Add titles
fig.update_layout(
    title=stockcode,
    yaxis_title='Stock Price (Rupee per Share)')

#Show
fig.show()

In [ ]:
data.